In [1]:
import quippy
import ase
from ase.calculators.calculator import all_changes

import logging
logging.basicConfig(level=logging.WARNING)

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [3]:
import os
import sys

In [4]:
sys.path.insert(0,"/home/es732/programs/function_modules/")
import mbx_functions

In [5]:
dirpath = os.getcwd()
print dirpath

/home/es732/programs/mbx_tests


In [6]:
from collections import OrderedDict

In [9]:
d_list=[0.1,0.01,0.001,0.0001,0.00001,0.000001]

In [65]:
version_list=["mbx_pbc","mbx_pbc_no_mww_pip","mbx_pbc_no_www_mww","mbx_pbc_no_ww_mw_www_mww",
              "mbx_pbc_no_ww_mw","mbx_pbc_no_ww","mbx_pbc_no_mw"]

In [66]:
dict_version_d_virial=OrderedDict([(version,
                                    OrderedDict([(d, [[np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN]])
                                                 for d in d_list]))
                                   for version in version_list])

In [67]:
dict_version_model_virial=OrderedDict([(version,[[np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN]])
                                   for version in version_list])

In [68]:
def calculate_numerical_virial_diag(pot, in_atoms, d=1e-2):
    ## modified from the function calculate_numerical_stress() at ase/calculators/calculator.py 
    """Calculate numerical stress using finite difference."""

    atoms=in_atoms.copy()
    stress = np.zeros((3, 3), dtype=float)

    cell = atoms.cell.copy()
    V = atoms.get_volume()
    for i in range(3):
        x = np.eye(3)
        x[i, i] += d
        atoms.set_cell(np.dot(cell, x), scale_atoms=True)
        atoms.set_calculator(pot)
        eplus = atoms.get_potential_energy()#force_consistent=True)
        pot.reset()

        x[i, i] -= 2 * d
        atoms.set_cell(np.dot(cell, x), scale_atoms=True)
        atoms.set_calculator(pot)
        eminus = atoms.get_potential_energy()#force_consistent=True)
        pot.reset()

        stress[i, i] = (eplus - eminus) / (2 * d * V)
        x[i, i] += d
    atoms.set_cell(cell, scale_atoms=True)
    virial= (-stress*atoms.get_volume())
    return virial

In [69]:
this_root="/home/es732/programs/mbx_tests/single_point/pbc/clathrate/"

In [70]:
this_struct="phase_clathrate_2x2x2"
this_at_tmp=quippy.Atoms(this_root+"11.74290/"+this_struct+".xyz")
dict_pot=mbx_functions.create_mbx_potential_variables(this_at_tmp)

## diagonal_virial=F, no PIPs

In [16]:
this_version=version_list[3]
print this_version

mbx_pbc_no_ww_mw_www_mww


In [17]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [18]:
at= this_at_tmp.copy()

In [19]:
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_version]

[[-1201.36578723     1.92256555     4.35904139]
 [    1.92256555 -1218.03530776     5.64420666]
 [    4.35904139     5.64420666 -1213.85989147]]


In [20]:
for tmp_d in [0.01,0.001,0.0001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

## diagonal_virial=F, no 3b PIPs

In [21]:
this_version=version_list[2]
print this_version

mbx_pbc_no_www_mww


In [22]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [23]:
at= this_at_tmp.copy()

In [24]:
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_version]

[[-2.49062583e+02  9.19533564e-01  1.05273100e-01]
 [ 9.19533564e-01 -2.47789626e+02 -2.06715905e+00]
 [ 1.05273100e-01 -2.06715905e+00 -2.46279058e+02]]


In [25]:
for tmp_d in [0.01,0.001,0.0001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

## diagonal_virial=F, full MBX

In [26]:
this_version=version_list[0]
print this_version

mbx_pbc


In [27]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [28]:
at= this_at_tmp.copy()

In [29]:
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_version]

[[-2.20669283e+02 -1.73001699e-01 -5.79919607e-01]
 [-1.73001699e-01 -2.20094017e+02 -2.49964680e-01]
 [-5.79919607e-01 -2.49964680e-01 -2.19885250e+02]]


In [30]:
for tmp_d in [0.1,0.01,0.001,0.0001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

## diagonal_virial=F, no 2b PIPs

In [50]:
this_version=version_list[4]
print this_version

mbx_pbc_no_ww_mw


In [51]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [52]:
at= this_at_tmp.copy()

In [35]:
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_version]

[[-1.17297249e+03  8.30030288e-01  3.67384868e+00]
 [ 8.30030288e-01 -1.19033970e+03  7.46140103e+00]
 [ 3.67384868e+00  7.46140103e+00 -1.18746608e+03]]


In [36]:
for tmp_d in [0.01,0.001,0.0001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

In [53]:
for tmp_d in [0.00001,0.000001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

## diagonal_virial=F, no ww 2b PIPs

In [72]:
this_version=version_list[5]
print this_version

mbx_pbc_no_ww


In [73]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [74]:
at= this_at_tmp.copy()

In [75]:
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_version]

[[-1118.07116494     1.49497428     3.35028786]
 [    1.49497428 -1135.65251189     6.57353867]
 [    3.35028786     6.57353867 -1132.19948817]]


In [76]:
for tmp_d in [0.01,0.001,0.0001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

In [77]:
for tmp_d in [0.00001,0.000001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

## diagonal_virial=F, no mw 2b PIPs

In [78]:
this_version=version_list[6]
print this_version

mbx_pbc_no_mw


In [79]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [80]:
at= this_at_tmp.copy()

In [81]:
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_version]

[[-2.75570605e+02 -8.37945692e-01 -2.56358785e-01]
 [-8.37945692e-01 -2.74781203e+02  6.37897674e-01]
 [-2.56358785e-01  6.37897674e-01 -2.75151845e+02]]


In [82]:
for tmp_d in [0.01,0.001,0.0001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

In [83]:
for tmp_d in [0.00001,0.000001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

In [84]:
version_list

['mbx_pbc',
 'mbx_pbc_no_mww_pip',
 'mbx_pbc_no_www_mww',
 'mbx_pbc_no_ww_mw_www_mww',
 'mbx_pbc_no_ww_mw',
 'mbx_pbc_no_ww',
 'mbx_pbc_no_mw']

## Errors in derivative virial / Virial from model (only diagonal)

### full model

In [62]:
this_version="mbx_pbc"
for tmp_d in [0.01,0.001,0.0001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    print "d =",tmp_d
    print np.diag((dict_version_d_virial[this_version][tmp_d] -dict_version_model_virial[this_version])/dict_version_model_virial[this_version])

d = 0.01
[0.01843925 0.01888588 0.01771213]
d = 0.001
[0.02160422 0.02203191 0.0208612 ]
d = 0.0001
[0.02163523 0.02206479 0.02089313]


### model without PIPs

In [61]:
this_version="mbx_pbc_no_ww_mw_www_mww"
for tmp_d in [0.01,0.001,0.0001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    print "d =",tmp_d
    print np.diag((dict_version_d_virial[this_version][tmp_d] -dict_version_model_virial[this_version])/dict_version_model_virial[this_version])

d = 0.01
[-0.00036129 -0.00033746 -0.00032154]
d = 0.001
[-3.61318902e-06 -3.37710712e-06 -3.21324275e-06]
d = 0.0001
[-3.64780990e-08 -3.38677207e-08 -3.20504887e-08]


### model without 2B PIPs

In [60]:
this_version="mbx_pbc_no_ww_mw"
for tmp_d in [0.01,0.001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
    print "d =",tmp_d
    print np.diag((dict_version_d_virial[this_version][tmp_d] -dict_version_model_virial[this_version])/dict_version_model_virial[this_version])

d = 0.01
[-0.00034597 -0.00032175 -0.00032353]
d = 0.001
[-3.24174712e-06 -3.45965736e-06 -3.29529788e-06]
d = 0.0001
[-3.33780810e-08 -3.53389607e-08 -4.75754578e-08]
d = 1e-05
[ 1.46731551e-09 -3.07620421e-10 -1.93714635e-09]
d = 1e-06
[ 8.95475914e-10  1.33834364e-08 -1.52305089e-08]


### model without 3B PIPs

In [59]:
this_version="mbx_pbc_no_www_mww"
for tmp_d in [0.01,0.001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
    print "d =",tmp_d
    print np.diag((dict_version_d_virial[this_version][tmp_d] -dict_version_model_virial[this_version])/dict_version_model_virial[this_version])

d = 0.01
[0.01622383 0.01666182 0.01578907]
d = 0.001
[0.01913916 0.01956941 0.01862555]
d = 0.0001
[0.01916878 0.0195986  0.01865408]
d = 1e-05
[3.70954921 3.80383185 3.82162832]
d = 1e-06
[3.7095492  3.80383192 3.82162826]


### model without 2B ww PIPs

In [85]:
this_version="mbx_pbc_no_ww"
for tmp_d in [0.01,0.001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
    print "d =",tmp_d
    print np.diag((dict_version_d_virial[this_version][tmp_d] -dict_version_model_virial[this_version])/dict_version_model_virial[this_version])

d = 0.01
[0.00389339 0.00391755 0.00369661]
d = 0.001
[0.00426649 0.00427242 0.004054  ]
d = 0.0001
[0.00427009 0.00427627 0.00405769]
d = 1e-05
[0.00427013 0.00427631 0.00405774]
d = 1e-06
[0.00427013 0.00427632 0.00405773]


### model without 2B mw PIPs

In [86]:
this_version="mbx_pbc_no_mw"
for tmp_d in [0.01,0.001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
    print "d =",tmp_d
    print np.diag((dict_version_d_virial[this_version][tmp_d] -dict_version_model_virial[this_version])/dict_version_model_virial[this_version])

d = 0.01
[-0.00250363 -0.00245759 -0.00245265]
d = 0.001
[-2.41412855e-05 -2.55328592e-05 -2.46206613e-05]
d = 0.0001
[-2.46579086e-07 -2.44570064e-07 -2.92782727e-07]
d = 1e-05
[ 9.10950493e-09 -8.63098351e-09 -1.35391094e-08]
d = 1e-06
[-1.19821476e-08  7.66451493e-08 -6.91632703e-08]


# test full potential with smaller deltas

## diagonal_virial=F, full MBX

In [36]:
this_version=version_list[0]
print this_version

mbx_pbc


In [37]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [38]:
at= this_at_tmp.copy()

In [39]:
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_version]

[[-2.20669283e+02 -1.73001699e-01 -5.79919607e-01]
 [-1.73001699e-01 -2.20094017e+02 -2.49964680e-01]
 [-5.79919607e-01 -2.49964680e-01 -2.19885250e+02]]


In [40]:
d_list=[0.1,0.01,0.001,0.0001,0.00001,0.000001,0.0000001]

In [41]:
dict_fullMBX_d_virial=OrderedDict([(d, [[np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN]])
                                                 for d in d_list])
                                   

In [42]:
for tmp_d in [0.1,0.01,0.001,0.0001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    dict_fullMBX_d_virial[tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

In [46]:
for tmp_d in [0.1,0.01,0.001,0.0001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    print (dict_fullMBX_d_virial[tmp_d]-dict_version_model_virial["mbx_pbc"])/dict_version_model_virial["mbx_pbc"]

[[-0.30704054 -1.         -1.        ]
 [-1.         -0.29877948 -1.        ]
 [-1.         -1.         -0.2894381 ]]
[[ 0.01843925 -1.         -1.        ]
 [-1.          0.01888588 -1.        ]
 [-1.         -1.          0.01771213]]
[[ 0.02160422 -1.         -1.        ]
 [-1.          0.02203191 -1.        ]
 [-1.         -1.          0.0208612 ]]
[[ 0.02163523 -1.         -1.        ]
 [-1.          0.02206479 -1.        ]
 [-1.         -1.          0.02089313]]


In [47]:
for tmp_d in [0.00001,0.000001,0.0000001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    dict_fullMBX_d_virial[tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

In [54]:
for tmp_d in [0.1,0.01,0.001,0.0001,0.00001,0.000001,0.0000001]:
    print "d=", tmp_d
    print "(virial_fd - virial) / virial"
    print (dict_fullMBX_d_virial[tmp_d]-dict_version_model_virial["mbx_pbc"])/dict_version_model_virial["mbx_pbc"]

d= 0.1
(virial_fd - virial) / virial
[[-0.30704054 -1.         -1.        ]
 [-1.         -0.29877948 -1.        ]
 [-1.         -1.         -0.2894381 ]]
d= 0.01
(virial_fd - virial) / virial
[[ 0.01843925 -1.         -1.        ]
 [-1.          0.01888588 -1.        ]
 [-1.         -1.          0.01771213]]
d= 0.001
(virial_fd - virial) / virial
[[ 0.02160422 -1.         -1.        ]
 [-1.          0.02203191 -1.        ]
 [-1.         -1.          0.0208612 ]]
d= 0.0001
(virial_fd - virial) / virial
[[ 0.02163523 -1.         -1.        ]
 [-1.          0.02206479 -1.        ]
 [-1.         -1.          0.02089313]]
d= 1e-05
(virial_fd - virial) / virial
[[ 0.02163557 -1.         -1.        ]
 [-1.          0.0220651  -1.        ]
 [-1.         -1.          0.0208935 ]]
d= 1e-06
(virial_fd - virial) / virial
[[ 0.02163555 -1.         -1.        ]
 [-1.          0.02206521 -1.        ]
 [-1.         -1.          0.02089343]]
d= 1e-07
(virial_fd - virial) / virial
[[ 0.02163646 -1.    

In [55]:
for tmp_d in [0.1,0.01,0.001,0.0001,0.00001,0.000001,0.0000001]:
    print "d=", tmp_d
    print "virial_fd "
    print (dict_fullMBX_d_virial[tmp_d])

d= 0.1
virial_fd 
[[-152.9148664    -0.           -0.        ]
 [  -0.         -154.3344403    -0.        ]
 [  -0.           -0.         -156.24208168]]
d= 0.01
virial_fd 
[[-224.73826005   -0.           -0.        ]
 [  -0.         -224.25068605   -0.        ]
 [  -0.           -0.         -223.77988539]]
d= 0.001
virial_fd 
[[-225.4366711    -0.           -0.        ]
 [  -0.         -224.94310895   -0.        ]
 [  -0.           -0.         -224.47232048]]
d= 0.0001
virial_fd 
[[-225.44351427   -0.           -0.        ]
 [  -0.         -224.95034554   -0.        ]
 [  -0.           -0.         -224.47934059]]
d= 1e-05
virial_fd 
[[-225.44358832   -0.           -0.        ]
 [  -0.         -224.95041419   -0.        ]
 [  -0.           -0.         -224.47942097]]
d= 1e-06
virial_fd 
[[-225.44358353   -0.           -0.        ]
 [  -0.         -224.95043758   -0.        ]
 [  -0.           -0.         -224.47940603]]
d= 1e-07
virial_fd 
[[-225.4437841    -0.           -0.        ]
 